## Problem tworzenia grafiku:
Znając stawki pracowników oraz wymaganą normę pracy danego dnia w tygodniu należy ułożyć grafik przy którym koszt zatrudnienia będzie minimalną funkcją celu.

### Ograniczeniami są:
- dostępność pracowników
- maksymalna liczba dni pracy jednego pracownika
- dzienna norma

### Dane wejściowe wyglądają następująco:

In [2]:
maxDaysForEmployee = 3
schedule = [2,1,1,1,3]
workersTime = {
    "Ania":   [1,1,1,1,1],
    "Stefan": [1,0,0,0,0],
    "Hektor": [1,1,1,0,0],
    "Olaf":   [1,1,1,1,1],
    "Lidia":  [0,0,0,1,1],
    "Piotr":  [1,1,1,0,0]
}
workersHRate = {
    "Ania":   100,
    "Stefan": 50,
    "Hektor": 60,
    "Olaf":   40,
    "Lidia":  110,
    "Piotr":  70
}

## Planowanie grafiku:

In [3]:
from  pulp import *

shop_model = LpProblem("Shop", LpMinimize)

pon = LpVariable.dicts('pon', [x for x in workersTime.keys() if workersTime[x][0] == 1], lowBound = 0, upBound = 1, cat = LpInteger)
wt = LpVariable.dicts('wt', [x for x in workersTime.keys() if workersTime[x][1] == 1], lowBound = 0, upBound = 1, cat = LpInteger)
sr = LpVariable.dicts('sr', [x for x in workersTime.keys() if workersTime[x][2] == 1], lowBound = 0, upBound = 1, cat = LpInteger)
czw = LpVariable.dicts('czw', [x for x in workersTime.keys() if workersTime[x][3] == 1], lowBound = 0, upBound = 1, cat = LpInteger)
pt = LpVariable.dicts('pt', [x for x in workersTime.keys() if workersTime[x][4] == 1], lowBound = 0, upBound = 1, cat = LpInteger)

shop_model += lpSum([workersHRate[w] * pon[w] for w in pon]) + lpSum([workersHRate[w] * wt[w] for w in wt]) + lpSum([workersHRate[w] * sr[w] for w in sr]) +  lpSum([workersHRate[w] * czw[w] for w in czw]) + lpSum([workersHRate[w] * pt[w] for w in pt]), "Total salary"

shop_model += lpSum([pon[w] for w in pon]) == schedule[0]
shop_model += lpSum([wt[w]  for w in wt])  == schedule[1]
shop_model += lpSum([sr[w]  for w in sr])  == schedule[2]
shop_model += lpSum([czw[w] for w in czw]) == schedule[3]
shop_model += lpSum([pt[w]  for w in pt])  == schedule[4]

for w in workersTime.keys():
    shop_model += lpSum([pon[e] for e in pon if w == e]) + lpSum([wt[e] for e in wt if w == e]) + lpSum([sr[e] for e in sr if w == e]) + lpSum([czw[e] for e in czw if w == e]) + lpSum([pt[e] for e in pt if w == e]) <= maxDaysForEmployee

shop_model.solve()

1

## Wyświetlenie obliczonego grafiku:

In [4]:
harmonogram = {"pon":[],"wt":[],"sr":[],"czw":[],"pt":[]}
for e in shop_model.variables():
    if e.varValue == 1.0:
        harmonogram[e.name.split("_")[0]].append(e.name.split("_")[1])

print("Harmonogram:")
for day in harmonogram.keys():
    print("",day)
    for empl in harmonogram[day]:
        print("  >",empl)

print("\nKoszt tygodniowy:", value(shop_model.objective), "zł")

Harmonogram:
 pon
  > Hektor
  > Stefan
 wt
  > Olaf
 sr
  > Hektor
 czw
  > Olaf
 pt
  > Ania
  > Lidia
  > Olaf

Koszt tygodniowy: 500.0 zł
